# Homework 5

by Maia Rosengarten <br/>
SID: 23572580<br/>
March 27, 2017<br/>
Kaggle Score:<br/>

In [1]:
import matplotlib
import numpy as np
from numpy import linalg as LA
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy.stats import logistic as sig
import pandas as pd

np.set_printoptions(threshold=np.nan)
from scipy import io
import sklearn
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import DictVectorizer as dv
from sklearn.preprocessing import Imputer as imp
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

# NP FNS

In [2]:
def vec(arr): # d b 1 --> (d, 1)
    return arr.reshape((arr.shape[0], 1))

In [3]:
def arr(vec): # 1 by d --> (d, )
    return vec.flatten()

# PREPROCESSING FNS

In [4]:
def vectorizeMatrix(data):
    '''
        Replaces all categorical variables that take on n possible values with
        n new columns in the design matrix. 
            e.g. If "Weather" takes on "sunny" or "rainy" would create two new features (columns)
                in the design matrix, one 'sunny' and the other 'rainy'. If sample point's weather is
                sunny, the new 'sunny' feature would take value 1, while 'rainy' would take 0.
        Args:
            data - a dataframe, the design matrix
    
    '''
    v = DictVectorizer(sparse=False)
    data = v.fit_transform(data.T.to_dict().values())
    return data

    

In [5]:
def fillMissingValuesWithAverage(designMatrix, isCat):
    '''
        Iterates over every value of every feature and replaces '?' (missing) with
        feature mean (if quantitative) and feature mode (if categorical)
        
        Args:
            designMatrix (nxd ndarray) - training data w n sample points and d features
            isSCat (bool) - boolean set to True if feature is categorical, False if quantitative
    
    '''

In [6]:
def fillMissingValuesWithKNN(designMatrix, k):
    '''
        Iterates over every value of every feature and replaces '?' (missing) with
        value returned by K Nearest Neighbors
        
        Args:
            designMatrix (nxd ndarray) - training data w n sample points and d features
            k (int) - determines the number of neighbors that will determine point's value
    
    '''


# UTIL FNS

In [7]:
def plotCostFN(costs):
    '''
        Plots cost function as a function of number of iterations
        Args:
            costs (ndarray) - lst of costs per iteration of gradient descent
    '''
   
    iters = [i for i in range(len(costs))]
    plt.plot(iters, costs)
    plt.xscale("log")
    plt.title("Cost Per Iteration of Gradient Descent")
    plt.xlabel('numIterations')
    plt.ylabel('cost')
    plt.show()

In [8]:
def generateCsv(aryPredictions, strCsvName):
    '''
    Writes predictions of testSet to csv file
    Args:
        aryPredictions (ndarray) - (nx1)-array of predictions given size n test (or valid) set
        strCsvName (str) - name of csv file to write to
    '''
    with open(strCsvName + '.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(aryPredictions)):
            writer.writerow([i, aryPredictions[i]])
    csvfile.close()

In [9]:
def readCSV(strFileName):
    '''
    Reads CSV file and returns data, labels tuple
    Args:
        strFileName (str) - path of csv file to read
    
    '''

In [10]:
def writeCleanedFeaturesToCSV(cleanedDesignMatrix):
    '''
        Writes cleaned version of design matrix (filled in missing values, processed categorical variables)
        to a CSV file to speed up work (don't need to preprocess every time you run code)
        
        Args: 
            cleanedDesignMatrix  
    
    '''

In [71]:
class DecisionTree:
    def __init__(self):
        self.root = None
#         self.attributeBag #<--ASK ABOUT THIS   
    
    def preProcess(X, y):
        '''
        Preprocesses the design matrix by filling in missing values with the feature average
        and vectorizing categorical features'''
        
    
    def train(self, train_data, train_labels, class0, height_cap):
        '''
            Sets the root of the decision tree to the root node of the resulting tree from buildTree.
            Args:
                train_data (ndarray) - cleaned training data
                train_lablels (ndarray) - cleaned training labels
                height_cap (int) - prevents tree from growing too large
        
        '''
        
        self.root = self.buildTree(train_data, train_labels, height_cap, class0)
    
    def buildTree(self, train_data, train_labels, height_cap, class0):
        '''
            Recursively grows a decision tree by constructing nodes. Using the impurity and segmenter methods, 
            attempts to find a configuration of nodes that best splits the input data. 
            This function figures out the split rules that each node should have and figures out 
            when to stop growing the tree and insert a leaf node.  
            
            This phase is training the data.
            
            Args:
                train_data (ndarray) - n by d matrix of cleaned training data
                train_labels (ndarray) - n by 1 vector of training labels
                height_cap (int) - caps the height of the tree to prevent it from growing too large
                
        '''
        #fix this stopping condition
        if (height_cap == 1):
            return Node()
        
        setSize = train_data.shape[0]
        lstUniqueLabels = set(train_labels)
        if len(lstUniqueLabels)==1: #isLeaf
            label = lstUniqueLabels[0]
            return Node(label=label)
        else:
            node = Node()
            bestSplitFeatIndex, bestSplitVal = node.chooseSplitRule(train_data, train_labels, class0)
            print('got best split feat/index for this node')
            leftIndices = [] 
            rightIndices = [] 
            lstSortedTupIndex2FeatVal = [i for i in sorted(enumerate(train_data[:, bestSplitFeatIndex]), key=lambda x: x[1])]
            
            lstSortedIndices = [i[0] for i in lstSortedTupIndex2FeatVal]
            lstSortedFeatVals = [i[1] for i in lstSortedTupIndex2FeatVal]
            
            indexSplitVal = lstSortedFeatVals.index(bestSplitVal)
            leftIndices = lstSortedIndices[:indexSplitVal+1]
            rightIndices = lstSortedIndices[indexSplitVal+1:]
            
            leftSetX = train_data[leftIndices, :]
            leftSetY = train_labels[leftIndices]
            
            rightSetX = train_data[rightIndices, :]
            rightSetY= train_labels[rightIndices]
            
            return Node(bestSplitFeatIndex, bestSplitVal,  self.buildTree(leftSetX, leftSetY, height_cap-1, class0), self.buildTree(rightSetX, rightSetY, height_cap-1, class0))                     
    
    
    
    def predictPoint(self, node, data_point):
        '''
            Given a data point, traverse the tree to find the best label to classify the data point. 
            Start at the root node and evaluate split rules as you traverse until you reach a leaf. 
            Choose that leaf nodes label as your output label.
            
            data_point (ndarray) - (dx1) single point to classify with d features 
        
        '''
        if (node.label is not None):
            return label
        if (data_point[node.splitFeatIndex] > node.splitVal):
            node.right.predict(data_point)
        else:
            node.left.predict(data_point) 
    
    
    
    def predict(self, dataMatrix):
        '''
            Given a matrix of test samples, predict labels.
            
            dataMatrix (ndarray) - (nxd) unseen samples with d dimensions 
        '''
        
        predictions = []
        for i in range(dataMatrix.shape[0]):
            predictions.append(self.predictPoint(self.root, dataMatrix[i]))
        return predictions
         


Go through all the spam data
Bag of words: have b words in vocabulary and vector is in R^b. vi = 1 if word i appears in document
if vocabulary is "ham", "spam", "sam" then you have your sentences: "sam likes spam": [0, 1, 1]...

When taking mutually exclusive sets, can overfit. Writing a better featurize.py
Take most commonly occurring words and make it an exclusive set. I get set of all words minus the stuff that the 
two classes have in common (like "the"). vocab of one class, vocab of two class. and can do set(v1) - set(v2) (but can overfit)...

if that can overfit, then can add other words from in between randomly. Or dimensionality reduction on total bag of words model.

In [69]:
class Node:
    def __init__(self, featureIndex=0, splitVal=None, leftChild=None, rightChild=None, label=None):
        self.splitFeatIndex = featureIndex 
        self.splitVal = splitVal
        self.left = leftChild
        self.right = rightChild 
        self.label = label #if set, this node is a leaf (mode), else is intermediary
    
    
    def chooseSplitRule(self, train_data, train_labels, class0):
        '''
            Finds the best split rule for a Node using the impurity measure and input data. 
            Exhaustively tries threshold values from the data and chooses the combination of 
            split feature and threshold with the lowest impurity value. 
           
            Args:
               train_data (ndarray) - cleaned train data 
               train_labels (ndarray) - train labels
            
            Returns (bestSplitFeatIndex, bestSplitValue)
                 
            
            If xi is quantitative, sort points in S by feature xi; 
            remove duplicates; [duplicate values, not points]
            try splitting between each pair of consecutive values.
            [We can radix sort the values in linear time, and if n is huge we should.]
            
            Clever Bit: As you scan sorted list from left to right, 
            you can update entropy in O(1) time per point! 
            [This is important for obtaining a fast tree-building time.] <--GO OVER THIS.
            
            Need to deal with case when a child is empty (skip that split)
    
        '''
        #<= split value --> left, > split value --> right
        
        #H before (PARENT NODE)
        currSamples = train_data[:, self.splitFeatIndex]
        currNodeSize = len(currSamples)
        print('currNodeSize ' + str(currNodeSize))
        currClass1Count = sum(train_labels) 
        currClass0Count = currNodeSize - currClass1Count
        currNodeEntropy = self.computeEntropy((currNodeSize, currClass0Count, currClass1Count))
        bestSplitIndex = self.splitFeatIndex
        bestSplitVal = self.splitVal
        bestInfoGain = -float("inf")
 
        #SET UP FOR SPLIT
        labelDictRight = {0: currClass0Count, 1: currClass1Count}
        labelDictLeft = {0:0, 1:0}
        sizeRightSet = currNodeSize
        sizeLeftSet = 0
        
        for featureIndex in range(train_data.shape[1]): 
            samples = train_data[:, featureIndex]
            tupSortedIndexVal = [i for i in sorted(enumerate(samples), key=lambda x:x[1])]
            sortedIndices = [i[0] for i in tupSortedIndexVal]
            sortedVals = [i[1] for i in tupSortedIndexVal] 
            
            lstUniqueVal, lstUniqueCounts = np.unique(sortedVals, return_counts=True)
            runningIndex = 0
            
            for i in range(len(lstUniqueVal)):
                runningIndex += (lstUniqueCounts[i] - 1)
                print('runningIndex: ' + str(runningIndex))
                tempSplitVal = lstUniqueVal[i]
                print('tempSplitValue ' + str(tempSplitVal))
                
                #update class count
                sizeLeftSet = runningIndex + 1
                sizeRightSet = currNodeSize - sizeLeftSet
                
                labelDictRight[1] = np.sum(train_labels[sortedIndices[runningIndex+1:]])
                labelDictRight[0] = sizeRightSet - labelDictRight[1]
                labelDictLeft[1] = np.sum(train_labels[sortedIndices[:runningIndex+1]])
                labelDictLeft[0] = sizeLeftSet - labelDictLeft[1]
                print('sides add up in right ' + str(labelDictRight[0] + labelDictRight[1] == sizeRightSet))
                print('sides add up in left ' + str(labelDictLeft[0] + labelDictLeft[1] == sizeLeftSet))
                print('sides add up to parent ' + str(sizeLeftSet + sizeRightSet == currNodeSize))
                
                tupSizeLeftClassCounts = (sizeLeftSet, labelDictLeft[0], labelDictLeft[1])
                tupSizeRightClassCounts = (sizeRightSet, labelDictRight[0], labelDictRight[1])
                print(tupSizeRightClassCounts)
                entropyAfterSplit = self.entropyAfterSplit(tupSizeLeftClassCounts, tupSizeRightClassCounts) 
                infoGain = currNodeEntropy - entropyAfterSplit
                print('info gain ' + str(infoGain))
                
#                 stopping condition: if number of points in class 0 or 1 contain total number of pionts.
                if infoGain > bestInfoGain:
                    bestInfoGain = infoGain
                    bestSplitVal = tempSplitVal
                    bestSplitIndex = featureIndex        
                    
                
#             sortedIndices = np.argsort(train_data[:, featureIndex]) 
#             uniqueFeatValLst = set(train_data[:, featureIndex])
#             sortedUniqueIndices = np.argsort(uniqueFeatValLst) 
#             print('sortedSampleIndices ' + str(sortedSampleIndices))  
            
#             lstVisitedIndices = set()
#             for sampleIndex in sortedUniqueIndices:
#                 print('\n sampleIndex ' + str(sampleIndex))
#                 tempSplitValue = train_data[sampleIndex][featureIndex]
               
                
#                 currIndex = sortedUniqueIndices.tolist().index(sampleIndex)
#                 if (currIndex != len(sortedSampleIndices) - 1):
# #                     print('currIndex ' + str(currIndex))
#                     nextIndex = currIndex + 1
#                     if (train_data[nextIndex][featureIndex] == tempSplitValue):
#                         continue
                
#                 print('tempSplitValue ' + str(tempSplitValue))
#                 leftSetIndices = sortedIndices[:sampleIndex+1]
#                 rightSetIndices = sortedIndices[sampleIndex+1:]
# #                 print('leftList: ' + str(leftSetIndices))
# #                 print('rightList: ' + str(rightSetIndices))
#                 sizeLeftSet = len(leftSetIndices)
#                 sizeRightSet = len(rightSetIndices)
# #                 print('sizeLeftSet ' + str(sizeLeftSet))
#                 print('sizeRightSet ' + str(sizeRightSet))
                
                #ask how to avoid iterating through everything each time ...
                
                # numInClass1 = sum(labels indices)
                # numInClass0 = 1 - numInClass1
#                 for elem in leftSetIndices:
#                     if train_labels[elem] == class0 and elem not in lstVisitedIndices: #need to figure out how to generalize class check + speed up.
#                         numClass0Left +=1
#                         numClass0Right -=1
#                         lstVisitedIndices.add(elem)
#                 print('sides add up: ' + str(numClass0Left + numClass0Right == currNodeClass0Count))
                    
#                 if sizeLeftSet==0 or sizeRightSet==0: #not sure about this
#                     continue
                
#                 print('sizeRightSet in splitRule ' + str(sizeRightSet))
#                 print('numClass0 in right in splitRule ' + str(numClass0Right))
#                 tupleSizeLeftSetClass0Count = (sizeLeftSet, numClass0Left)
#                 tupleSizeRightSetClass0Count = (sizeRightSet, numClass0Right)
#                 entropyAfterSplit = self.entropyAfterSplit(tupleSizeLeftSetClass0Count, tupleSizeRightSetClass0Count) 
# #                 print('entropy after split ' + str(entropyAfterSplit))
#                 infoGain = currNodeEntropy - entropyAfterSplit
# #                 print('info gain ' + str(infoGain))
                
# #                 stopping condition: if number of points in class 0 or 1 contain total number of pionts.
#                 if infoGain > bestInfoGain:
# #                     print('update best split val and indices')
#                     bestInfoGain = infoGain
#                     bestSplitVal = tempSplitValue
#                     bestSplitIndex = featureIndex        
                
        
        print('bestSplit Index: ' + str(bestSplitIndex) + ', bestSplitVal: ' + str(bestSplitVal))
        
        self.splitFeatIndex = bestSplitIndex 
        self.splitVal = bestSplitVal
        
        return bestSplitIndex, bestSplitVal
            
        '''
        
        1. Only have indices of the sample points, don't throw the sample points around.
        2. lst holding the left vs the right points
        3. Don't need to iterate through the lists to figure out if class 1 or 0...do that once when initialiez
            - because sorting on index, cardinality of 
        4. entropy: compute entropy separately: all elements
        

        
        '''
        
              
    def entropyAfterSplit(self, tupSizeLeftClassCounts, tupSizeRightClassCounts):
        '''
            A method that takes in the result of a split: two histograms
            that count the frequencies of labels on the ”left” and ”right” side of that split.
            Calculates and outputs a scalar value representing the impurity (”badness”) of the 
            specified split on the input data. In lecture, this was refered to as H_after where H
            is the function that computes entropy.
            
            Args:
                left_label_hist (dict) - dictionary of label values to frequencies on the left side of the split
                right_label_hist (dict) - dictionary of label values to frequencies on the right side of the split
        
        
        impurity = calculate entropy  - impurity is within a certain node; entropy is accross nodes
        - purity: the state of something. 
        
        Class A or Not A (binary)
        
        
        '''
        sizeLeftSet = tupSizeLeftClassCounts[0]
        sizeRightSet = tupSizeRightClassCounts[0]
        entropyLeft = self.computeEntropy(tupSizeLeftClassCounts)
        entropyRight = self.computeEntropy(tupSizeRightClassCounts)
        entropyAfterSplit = (entropyLeft * sizeLeftSet + entropyRight * sizeRightSet)/ (sizeLeftSet + sizeRightSet)
        return entropyAfterSplit    
    
    
    def computeEntropy(self, tupleSizeSetClassCounts):
        '''
            Compute the entropy of a set (assume 2 classes). 
            
            Args:
                tupleSetSizeClassSize (tup) - (cardinality of set, cardinality of class 0 in set) 
        
        
        '''
        sizeSet, countClass0, countClass1 = tupleSizeSetClassCounts
        probClass0 = countClass0/sizeSet
        if (probClass0 == 1 or probClass0 == 0):
            return 0
        else:
            print('probClass0 ' + str(probClass0))
            entropy = (- probClass0 * np.log2(probClass0)) - ((1 - probClass0) * np.log2(1 - probClass0))
        return entropy

In [21]:
class RandomForest:
    def init(self, attributeBag, dataBag):
        self.attributeBag = attributeBag #subset of features to train each node on in different trees
        self.dataBag = dataBag #subset of data to train each tree on 

# LOAD DATASET

In [14]:
#SPAM DATA

spamData = sp.io.loadmat("dist/spam_data.mat")
spamData.keys()
spamTrainX = spamData['training_data']
spamTrainY = spamData['training_labels']
spamTest = spamData['test_data']

# spamTest = sklearn.preprocessing.normalize(spamTest.astype(float))
# spamTrainX = sklearn.preprocessing.normalize(spamTrainX.astype(float))


combined = np.hstack([spamTrainX, spamTrainY.T])
np.random.shuffle(combined)
combined = imp().fit_transform(combined)

spamTrainX = combined[:, :-1]
spamTrainY = combined[:, -1]

#maybe add a bias feature

trainX, validX, trainY, validY = train_test_split(spamTrainX, spamTrainY, test_size=.1, random_state=42)
print("")

In [70]:
height_cap = 2
class0 = 0
decisionTree = DecisionTree()
decisionTree.train(trainX, trainY, class0, height_cap)
decisionTree.root
print(decisionTree.root.splitFeatIndex)

currNodeSize 21331
probClass0 0.491866297876
sizeRightSet 21331
lstUniqueVal [  0.   1.   2.   3.   4.   6.   7.   8.  11.  13.]
lstUniqueCounts [20973   299    42     6     6     1     1     1     1     1]
runningIndex: 20972
tempSplitValue 0.0
sizeLeft 20973
sizeRight 358
sides add up in right True
sides add up in left True
sides add up to parent True
(358, 23.0, 335.0)
probClass0 0.499165593859
probClass0 0.0642458100559
entropy after split 0.988989555102
info gain 0.0108195474081
runningIndex: 21270
tempSplitValue 1.0
sizeLeft 21271
sizeRight 60
sides add up in right True
sides add up in left True
sides add up to parent True
(60, 3.0, 57.0)
probClass0 0.493112688637
probClass0 0.05
entropy after split 0.997856283925
info gain 0.00195281858552
runningIndex: 21311
tempSplitValue 2.0
sizeLeft 21312
sizeRight 19
sides add up in right True
sides add up in left True
sides add up to parent True
(19, 0.0, 19.0)
probClass0 0.492304804805
entropy after split 0.998938561684
info gain 0.000870

NameError: name 'buildTree' is not defined

In [16]:
predictions = decisionTree.predict(validX)

TypeError: unorderable types: float() > NoneType()

In [34]:
accuracy = accuracy_score(validY, predictions)
print(accuracy)

NameError: name 'predictions' is not defined

In [209]:
#census data
censusTrainData = pd.read_csv('hw5_census_dist/train_data.csv')
censusTestData = pd.read_csv('hw5_census_dist/test_data.csv')

#one hot encoding
censusTrainData = vectorizeMatrix(censusTrainData)
censusTestData = vectorizeMatrix(censusTestData)

#missing values
censusTrainData = imp().fit_transform(censusTrainData)

censusTestData = imp().fit_transform(censusTestData)

np.random.shuffle(censusTrainData)
np.random.shuffle(censusTestData)

censusTrainX = censusTrainData[:, :-1]
censusTrainY = censusTrainData[:, -1]
censusTestX = censusTestData[:, :-1]
censusTestY = censusTestData[:, -1]


In [207]:
#titanic data
titanicTrainData = pd.read_csv('hw5_titanic_dist/titanic_training.csv')
titanicTestData =  pd.read_csv('hw5_titanic_dist/titanic_testing_data.csv')

#one hot encoding
titanicTrainData = vectorizeMatrix(titanicTrainData)
titanicTestData = vectorizeMatrix(titanicTestData)

#missing values
titanicTrainData = imp().fit_transform(titanicTrainData)

titanicTestData = imp().fit_transform(titanicTestData)


np.random.shuffle(titanicTrainData)
np.random.shuffle(titanicTestData)

titanicTrainX = titanicTrainData[:, :-1]
titanicTrainY = titanicTrainData[:, -1]
titanicTestX = titanicTestData[:, :-1]
titanicTestX = titanicTestData[:, -1]
